In [1]:
# !pip3 install torch
# !pip3 install torchvision
# !pip3 install matplotlib
# !pip3 install torchmetrics

import numpy as np
import torch
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision
from torchvision import datasets
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
print("Success")

Success


## OverFitting
- 학습 데이터 셋에 너무 Fitting 되 일반화 하지 못하는 상황
- 데이터가 작을 수록 Overfitting이 될 확률이 높다.
- Feature가 많을 수록 Overfitting이 될 확률이 높다.
- Validation set의 loss가 가장 작은 모델 선택
- Regularization을 통해 Overfitting을 막는다.

## Regularization
- Early stopping
- Rdeucing Network Size
- Weight Decay
- Droptout
- Batch Noramlization

In [2]:
x_train = torch.FloatTensor([[1, 2, 1],
                             [1, 3, 2],
                             [1, 3, 4],
                             [1, 5, 5],
                             [1, 7, 5],
                             [1, 2, 5],
                             [1, 6, 6],
                             [1, 7, 7]
                            ])
y_train = torch.LongTensor([2, 2, 2, 1, 1, 1, 0, 0])

x_test = torch.FloatTensor([[2, 1, 1], [3, 1, 2], [3, 3, 4]])
y_test = torch.LongTensor([2, 2, 2])



In [3]:
class SoftmaxClassification(nn.Module):
  def __init__(self):
    super().__init__()
    self.linear = nn.Linear(3,3)

  def forward(self, x):
    return self.linear(x)

model = SoftmaxClassification()

In [4]:
optimizer = optim.SGD(model.parameters(), lr = 0.1)

In [5]:
def train(model, optimizer, x_train, y_train):
  np_epochs = 20

  for epoch in range(1, np_epochs+1):

    output = model(x_train)
    cost = F.cross_entropy(output, y_train)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    print(f"Epoch : {epoch}/{np_epochs}, Loss : {cost.item()}")
    

In [6]:
def test(model, x_test, y_test):

  prediction = model(x_test)
  predicted_classes = prediction.max(dim = 1)[1]

  correct_count = (predicted_classes == y_test).sum().item()
  cost = F.cross_entropy(prediction, y_test)
  print(f"ACC : {correct_count / len(y_test) * 100}, Loss : {cost}")

In [7]:
train(model, optimizer, x_train, y_train)

Epoch : 1/20, Loss : 3.687032699584961
Epoch : 2/20, Loss : 1.4187272787094116
Epoch : 3/20, Loss : 1.1026562452316284
Epoch : 4/20, Loss : 1.0703225135803223
Epoch : 5/20, Loss : 1.058597207069397
Epoch : 6/20, Loss : 1.0481348037719727
Epoch : 7/20, Loss : 1.0385676622390747
Epoch : 8/20, Loss : 1.0296778678894043
Epoch : 9/20, Loss : 1.0212453603744507
Epoch : 10/20, Loss : 1.0131970643997192
Epoch : 11/20, Loss : 1.0054500102996826
Epoch : 12/20, Loss : 0.9979684948921204
Epoch : 13/20, Loss : 0.9907166957855225
Epoch : 14/20, Loss : 0.9836755394935608
Epoch : 15/20, Loss : 0.976827085018158
Epoch : 16/20, Loss : 0.9701598286628723
Epoch : 17/20, Loss : 0.9636633396148682
Epoch : 18/20, Loss : 0.9573297500610352
Epoch : 19/20, Loss : 0.9511518478393555
Epoch : 20/20, Loss : 0.945123553276062


In [9]:
test(model, x_test, y_test)

ACC : 66.66666666666666, Loss : 0.7797987461090088


## Data Preprocessed
- 결과 값이 (m,2)이고
  [ 1000, 0.1]
  [999, 0.2]..

- 일때 네트워크는 col=0 인 데이터에 치중에서 학습 된다.
- 이떄 정규화를 해주면 모든 데이터가 비슷한 분포로 학습 된다. 